<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Models-and-Results" data-toc-modified-id="Models-and-Results-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Models and Results</a></span><ul class="toc-item"><li><span><a href="#Basic-SIR" data-toc-modified-id="Basic-SIR-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Basic SIR</a></span></li><li><span><a href="#Spatial-SIR" data-toc-modified-id="Spatial-SIR-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Spatial SIR</a></span></li><li><span><a href="#SIR-with-varition-on-Parameters" data-toc-modified-id="SIR-with-varition-on-Parameters-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>SIR with varition on Parameters</a></span></li><li><span><a href="#SEIR" data-toc-modified-id="SEIR-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>SEIR</a></span></li><li><span><a href="#Fit-to-Data" data-toc-modified-id="Fit-to-Data-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Fit to Data</a></span></li></ul></li><li><span><a href="#Discussion" data-toc-modified-id="Discussion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Discussion</a></span><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Conclusion</a></span></li><li><span><a href="#Limitation" data-toc-modified-id="Limitation-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Limitation</a></span></li><li><span><a href="#Further-Investigation" data-toc-modified-id="Further-Investigation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Further Investigation</a></span></li></ul></li><li><span><a href="#Bibliography" data-toc-modified-id="Bibliography-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Bibliography</a></span></li></ul></div>


In addition to the code you will write and simulations you will run, you should prepare a final report, which should be similar in length/format to a conference paper or short journal article. The purpose of the report is to explain what you investigated, how you investigated your topics (e.g. model/implementation), and why your investigation was interesting (what should a reader take away from your work).

This can either be a PDF or a file which can be viewed in a web browser (please don't just use GitHub to host a notebook or markdown file - generate a static html web page). Put your report in the doc/final folder in the repository, which should contain any files and figures, as well as instructions to generate the PDF or html files (if they are generated from LaTeX, Markdown, etc.) If your extensions include an interactive visualization, find a good way to display this online. The purpose of the report is to explain what you investigated, how you investigated your topics, and why your investigation was interesting.

Your report should the following sections. 0. Abstract (1 paragraph summary)

A brief introduction to the SIR model, the notation you will use, and the different variations you investigate.
Sections for the basic SIR model (also in midterm checkpoint), the spatial SIR model, and your variations. Include any relevant implementation details as well as your results and discussion.
A conclusion summarizing your results, discussing limitations of your models, and pointing out interesting directions for further investigation
A bibliography
Please limit your report to 10 pages (excluding bibliography) with 1-inch margins (these margins allow for a lot of content per page). You can put additional figures/results in appendices (not counting to 10 page limit) if you would like, but it is perfectly fine to choose one or two figures per experiment that best illustrate your point in the main text.

# Introduction

# Models and Results

## Basic SIR

## Spatial SIR

### PDE

The PDEs we want to solve are:
\begin{align}
  \frac{\partial s(\mathbf{x},t)}{\partial t} &= -b\cdot s(\mathbf{x},t) \cdot i(\mathbf{x},t) + p\cdot \Delta s(\mathbf{x},t) \\
  \frac{\partial r(\mathbf{x},t)}{\partial t} &= k\cdot i(\mathbf{x},t) + p\cdot \Delta r(\mathbf{x},t) \\
  \frac{\partial i(\mathbf{x},t)}{\partial t} &= b\cdot s(\mathbf{x},t)\cdot i(\mathbf{x},t) - k\cdot i(\mathbf{x},t) + p\cdot \Delta i(\mathbf{x},t)
\end{align}

We want to use finite differencing to transform these PDEs into systems of ODEs and then use `solve_ivp` to get the solutions. We first discretize $[0,1]$ on the $x$-axis to $0 = x_0 < x_1 < \cdots < x_M = 1$ where $x_i - x_{i-1} = \frac{1}{M} := h$ for all $i$ and similarly for $y_0,\ldots,y_M$.

Consider the first PDE on $s(\mathbf{x},t)$. The Laplacian at $(x_j,y_k)$ becomes
\begin{align*}
  \Delta s(x_j,y_k,t) &= \frac{\partial^2 s(x_j,y_k,t)}{\partial x^2} + \frac{\partial^2 s(x_j,y_k,t)}{\partial y^2} \\
  &= \frac{s(x_{j+1},y_k,t) - 2s(x_j,y_k,t) + s(x_{j-1},y_k,t)}{h^2} + \frac{s(x_j,y_{k+1},t) - 2s(x_j,y_k,t) + s(x_j,y_{k-1},t)}{h^2}
\end{align*}

Define $s_{j,k}(t) = s(x_j,y_k,t)$. Then the PDE becomes a system of ODEs:
\begin{align}
  \frac{ds_{j,k}(t)}{dt} = -b\cdot s_{j,k}(t)\cdot i_{j,k}(t) + \frac{p}{h^2} [s_{j+1,k}(t) + s_{j-1,k}(t) + s_{j,k+1}(t) + s_{j,k-1}(t) - 4s_{j,k}(t)], \quad \forall j,k = 1,\ldots,M
\end{align}

Similarly, we define $i_{j,k}(t)$ and $r_{j,k}(t)$ and the other two PDEs become
\begin{align}
  \frac{dr_{j,k}(t)}{dt} &= k\cdot i_{j,k}(t) + \frac{p}{h^2} [r_{j+1,k}(t) + r_{j-1,k}(t) + r_{j,k+1}(t) + r_{j,k-1}(t) - 4r_{j,k}(t)], \quad \forall j,k = 1,\ldots,M \\
  \frac{di_{j,k}(t)}{dt} &= b\cdot s_{j,k}(t)\cdot i_{j,k}(t) - k\cdot i_{j,k}(t) + \frac{p}{h^2} [i_{j+1,k}(t) + i_{j-1,k}(t) + i_{j,k+1}(t) + i_{j,k-1}(t) - 4i_{j,k}(t)], \quad \forall j,k = 1,\ldots,M
\end{align}

To write these in matrix form, we need to define the second-order finite differencing matrix
\begin{align*}
  D = \begin{bmatrix}
    -2 & 2 & 0 \\
    1 & -2 & 1 & 0\\
    0 & 1 & -2 & 1 & 0 \\
    & & \ddots & \ddots & \ddots \\
    & & & 0 & 1 & -2 & 1 \\
    & & & & 0 & 2 & -2
  \end{bmatrix}
\end{align*}
where the empty entries are all 0. Then we can write the three system of ODEs as
\begin{align*}
  \frac{ds(t)}{dt} &= -b\cdot s(t)\times i(t) + \frac{p}{h^2}(Ds(t) + s(t)D^\top) \\
  \frac{di(t)}{dt} &= k\cdot i(t) + \frac{p}{h^2}(Di(t) + i(t)D^\top) \\
  \frac{dr(t)}{dt} &= b\cdot s(t)\times i(t) - k\cdot i(t) + \frac{p}{h^2}(Dr(t) + r(t)D^\top)
\end{align*}
where $s(t),i(t),r(t)$ are the $M\times M$ matrix-valued functions and $s(t)\times i(t)$ denotes the entry-wise multiplication.

## SIR with varition on Parameters

## SEIR

## Fit to Data

# Discussion

## Conclusion

## Limitation 

## Further Investigation

# Bibliography 